Load data and inputs

In [6]:
import robustsp as rsp
import numpy as np
import scipy.io
import pkg_resources

# Read image of sqares (20 x 20 pixels)
path = pkg_resources.resource_filename('robustsp', 'data/images.mat') 
# contains the vectors y20 (clean data) and y20n (noisy data)
img = scipy.io.loadmat(path,struct_as_record=False)
y20 = img['y20']
y20n = img['y20n']
n = len(y20)

scaledata1 = lambda x: 3*(x-np.min(x)) / (np.max(x) - np.min(x))
scaledata2 = lambda x: 3*(x-np.min(x,axis=1)[:,None]) / (np.max(x,axis=1) - np.min(x,axis=1))[:,None]

lambda2 = 340
lambda1 = 124

L = 20 # Grid size
Blas20n, stats = rsp.enetpath(y20n, np.eye(n), 1, L,10**-3,False)
Blas20n = Blas20n[:,1:] # Get rid of the all-zeros first column 

ero = scaledata2(Blas20n.T).T - y20
MSElasso = np.min(np.sum(ero**2,axis=0))
indx = np.argmin(np.sum(ero**2,axis=0)) # should b 6

Blas = Blas20n[:,indx] # the best lasso solution

In [34]:
def rankflasso(yx,Xx,lambda1,lambda2,b0=None,printitn=0):

    y = np.array(yx)
    X = np.array(Xx)
    n,p = X.shape

    intcpt = False

    if b0 is None:
        # Note that underdetermined systems have indefinite solutions, numpy handles it differently
        # than matlab
        b0 = np.linalg.lstsq(np.hstack((np.ones((n,1)),X)),y[:,None])[0] 
        b0 = b0[1:]

    B = np.repeat(np.arange(1,n+1)[:,None],n,axis=1)
    A = np.array(B.T)
    a = A[A<B]
    b = B[A<B]

    D = -1*np.eye(p-1) + np.diag(np.ones(p-2),k=1)
    
    onev = np.append(np.zeros(p-2),1)
    D = np.hstack((D,onev[:,np.newaxis]))

    ytilde = np.append(y[a-1]-y[b-1],
                        np.zeros(p-1)
                       )
    
    Xtilde = np.vstack((X[a-1,:]-X[b-1,:],lambda2*D))

    if printitn > 0:
        print("rankflasso: starting iterations\n")

    b, iter = rsp.ladlasso(ytilde,Xtilde,lambda1,b0,intcpt,printitn)

    return b, iter

y = y20n[:40]
B20 = Blas[:40]
B1 = rankflasso(y,np.eye(40),lambda1,lambda2,B20,1)[0]
B1

rankflasso: starting iterations



array([-3.57168858e-10, -4.30626620e-10, -4.83506782e-10, -5.57444962e-10,
       -6.79576417e-10, -7.62340322e-10, -7.82143073e-10, -7.07016573e-10,
       -9.85455756e-10, -1.05760012e-09, -1.03685180e-09, -9.03150034e-10,
        3.70226543e-07,  3.70331649e-07,  3.70260146e-07,  7.32275946e-08,
        7.05884932e-08,  6.92147121e-08,  6.71635294e-08,  6.64807936e-08,
        1.26414923e-09,  1.08977780e-09,  9.68450064e-10,  9.10914143e-10,
       -8.75101653e-10, -1.10953597e-09, -1.18878148e-09, -1.20988384e-09,
       -1.07184366e-09, -9.81213569e-10, -7.52928523e-10, -4.43129422e-10,
        2.07896559e-06,  2.07914131e-06,  2.07921594e-06,  2.07912050e-06,
        2.07346304e-06,  2.07351985e-06,  2.07350813e-06,  2.07351957e-06])

In [32]:
B1mat = scipy.io.loadmat('b1')['B1']
np.isclose(B1mat,B1) # false
np.isclose(B1mat,B1,atol=1e-3) # false
np.sum(np.abs(B1mat-B1))

131837.25240483528

rankflasso: starting iterations



array([ 1.07358143e-01, -1.76110494e-02, -6.50804319e-02, -3.81822716e-02,
       -1.08637574e-09, -1.43932305e-09, -1.88311411e-09,  1.94959778e+00,
       -2.77026624e-09, -1.94435468e-09, -2.15879314e-09, -2.34235056e-09,
        2.12451209e+00,  2.06458051e+00,  2.24353605e+00,  9.72877771e-01,
        5.25812258e-01,  9.88553923e-01,  1.09868272e+00,  1.32490883e+00,
       -1.49130808e-02,  3.60886290e-02, -3.76907643e-02,  1.94959778e+00,
       -1.10171496e-09, -2.52591170e-09, -1.91375484e-09, -2.63303052e-09,
       -1.11416369e-01, -2.17415341e-09, -1.73007429e-09, -1.03996900e-09,
        2.16368503e+00,  1.85818194e+00,  1.84037582e+00,  1.21254780e+00,
        7.60367824e-01,  1.22621378e+00,  1.03521161e+00,  1.12124702e+00,
        5.06814379e-03, -1.73007114e-09, -9.27137860e-02,  1.23941165e-01,
       -1.46992984e-09, -2.25077632e-09, -2.12821660e-09, -7.01189162e-10,
       -2.14349286e-09, -2.81576390e-09, -2.57184701e-09, -1.65154438e-05,
       -2.27017606e-02, -

Inputs:

y = [1 2 3]

X =

     1     2
     4     5
     6     7

lambda1 = 0.1

lambda2 = 0.2

Output:

b =

    0.2000
    0.2000


i =

        2000


In [30]:
import robustsp as rsp
import numpy as np

y = np.array([1,2,3])
X = np.array(([1,2],[4,5],[6,7]))
lambda1 = 0.1
lambda2 = 0.2
printitn = 0
b0 = None

#######################################################################

n,p = X.shape

intcpt = False

if b0 is None:
    b0 = np.linalg.solve(np.hstack((np.ones((n,1)),X)),y)
    b0 = b0[1:]


B = np.repeat(np.arange(1,n+1)[:,None],n,axis=1)
A = np.copy(B.T)
a = A[A<B]
b = B[A<B]

D = -1*np.eye(p-1)
D[p:p:(p-1)**2]=1
onev = np.append(np.zeros(p-2),1)
D = np.hstack((D,onev[:,np.newaxis]))

ytilde = np.append(y[a-1]-y[b-1],
                    np.zeros(p-1)
                   )
Xtilde = np.vstack((X[a-1,:]-X[b-1,:],lambda2*D))

if printitn > 0:
    print("rankflasso: starting iterations\n")

b, iter = rsp.ladlasso(ytilde,Xtilde,lambda1,b0,intcpt,printitn)

return b, iter


(array([0.19999998, 0.19999998]), 1999)

In [1]:
import robustsp as rsp
import numpy as np
y = np.array([1,2,3])
X = np.array(([1,2],[4,5],[6,7]))
b,i = rsp.rankflasso(y,X,0.1,0.2)
b,i

(array([0.19999998, 0.19999998]), 1999)